# Decoding with SpaceNet  
https://nilearn.github.io/modules/generated/nilearn.decoding.SpaceNetClassifier.html#nilearn.decoding.SpaceNetClassifier

In [ ]:
import numpy as np
import pandas as pd
import os
import numpy as np
import nilearn
import glob
from nilearn.image import index_img
#from nilearn.datasets import load_mni152_brain_mask


### Load the Chocolate data 
- Starting with W1 (133 Subjects) 
- The first part we are going to try to decode on 'app' vs 'h2O'

In [ ]:
#mask_img = load_mni152_brain_mask()

stim = os.path.join('/projects','niblab','scripts','nilean_stuff','label_all_sub.csv')
behavioral = pd.read_csv(stim, sep=",")
conditions =  behavioral['label']
condition_mask = conditions.isin(['app', 'H2O'])

img_mask="/projects/niblab/nilearn_projects/power_roimask_4bi.nii.gz"

# Split data into train and test samples, using the chunks
condition_mask_train = (condition_mask) & (behavioral['sub'] == 0)
condition_mask_test = (condition_mask) & (behavioral['sub']  == 1)


# Apply this sample mask to X (fMRI data) and y (behavioral labels)
# Because the data is in one single large 4D image, we need to use
# index_img to do the split easily
func_filenames='/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/group_ana/w1_imagine_all.nii.gz'
X_train = index_img(func_filenames, condition_mask_train)
X_test = index_img(func_filenames, condition_mask_test)
y_train = conditions[condition_mask_train]
y_test = conditions[condition_mask_test]

# Compute the mean epi to be used for the background of the plotting
from nilearn.image import mean_img
background_img = mean_img(func_filenames)

### Fit SpaceNet with a Graph-Net penalty
SpaceNetClassifier 
    - adding mask
    - need to look at alphas and cv

In [ ]:
from nilearn.decoding import SpaceNetClassifier
# Fit model on train data and predict on test data
decoder = SpaceNetClassifier(memory_level=2, n_jobs=4,
                             mask=img_mask, standardize=True,
                             memory="nilearn_cache", penalty='graph-net')
decoder.fit(X_train, y_train)
y_pred = decoder.predict(X_test)
accuracy = (y_pred == y_test).mean() * 100.
print("Graph-net classification accuracy : %g%%" % accuracy)

Graph-net classification accuracy : 52.9925%

### Visualization of Graph-net weights

In [ ]:
from nilearn.plotting import plot_stat_map, show
coef_img = decoder.coef_img_
plot_stat_map(coef_img, background_img,
              title="graph-net: accuracy %g%%" % accuracy,
              cut_coords=(-52, -5), display_mode="yz")

# Save the coefficients to a nifti file
coef_img.to_filename('haxby_graph-net_weights.nii')

### Now we are going to fit SpaceNet with a TC-l1 penalty 

In [ ]:
decoder = SpaceNetClassifier(memory="nilearn_cache", penalty="tv-l1")
decoder.fit(X_train, y_train)
y_pred = decoder.predict(X_test)
accuracy = (y_pred == y_test).mean() * 100
print("TV-l1 classification accuracy : %g%%" % accuracy)